<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-and-Analyse" data-toc-modified-id="Load-and-Analyse-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load and Analyse</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Structuring-the-data" data-toc-modified-id="Structuring-the-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Structuring the data</a></span></li><li><span><a href="#Text-Preprocessing" data-toc-modified-id="Text-Preprocessing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Text Preprocessing</a></span></li></ul></li><li><span><a href="#Word-Embeddings" data-toc-modified-id="Word-Embeddings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Word Embeddings</a></span></li><li><span><a href="#Feature-Engineering-for-Traditional-ML" data-toc-modified-id="Feature-Engineering-for-Traditional-ML-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Feature Engineering for Traditional ML</a></span></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Traditional-ML-Models" data-toc-modified-id="Traditional-ML-Models-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Traditional ML Models</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Naive-Bayes" data-toc-modified-id="Naive-Bayes-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Naive Bayes</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Random Forest</a></span></li></ul></li><li><span><a href="#Deep-Learning-Models" data-toc-modified-id="Deep-Learning-Models-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Deep Learning Models</a></span><ul class="toc-item"><li><span><a href="#Using-Word2Vec-Models-created" data-toc-modified-id="Using-Word2Vec-Models-created-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Using Word2Vec Models created</a></span></li><li><span><a href="#Dense-Neural-Network" data-toc-modified-id="Dense-Neural-Network-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Dense Neural Network</a></span></li><li><span><a href="#Convolutional-Neural-Network" data-toc-modified-id="Convolutional-Neural-Network-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Convolutional Neural Network</a></span></li></ul></li></ul></li></ul></div>

In [136]:
import pandas as pd
import numpy as np

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
#nltk.download('wordnet')

import gensim
from gensim.models.word2vec import Word2Vec

from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

from tensorflow.keras.layers import Embedding
from tensorflow.python.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns

## Load and Analyse

In [55]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [57]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
data.shape

(1989, 27)

In [5]:
data.Date.dtype

dtype('O')

In [6]:
data.Date = pd.to_datetime(data.Date)

In [7]:
data.Date.dtype

dtype('<M8[ns]')

In [8]:
data.Date.min()

Timestamp('2008-08-08 00:00:00')

In [9]:
data.Date.max()

Timestamp('2016-07-01 00:00:00')

In [10]:
(data.Date.max() - data.Date.min())

Timedelta('2884 days 00:00:00')

In [11]:
data.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [12]:
data.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [13]:
null_indices = pd.isnull(data).any(axis = 1).nonzero()[0]
null_indices

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


array([277, 348, 681], dtype=int64)

In [14]:
data.iloc[null_indices]

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
277,2009-09-15,1,b'The Church of Scientology won\'t be dissolve...,b'New virus from rats can kill 80 per cent of ...,b'The gruesome spectacle of dolphins being sla...,b'The End of Innocence in Afghanistan: \'The G...,b'France approves Internet piracy bill',b'The Rural Doctors Association says right now...,b'Al Jazeera English - Africa - Shabab to aven...,"b""How Sri Lanka governs through detentions - S...",...,b'In an equine echo of the controversy surroun...,b'UPDATE: 5-New York homes raided in terrorism...,b'Population Growth Impeding Progress on the M...,b'Global Population to Reach 7 Billion by 2011',b'Government Funded Feminist Porn ',b'Can someone enlighten me re:Holy Land disput...,b'Human Rights Watch official suspended for co...,NaN,NaN,NaN
348,2009-12-24,1,b'Woman knocks down Pope Benedict at Christmas...,b'Ugandan President Museveni says he will bloc...,"b""Venezuela's Chavez threatens to kick out car...",b'Woman who knocked down pope had pasta in her...,"b'450 people from 21 countries, including 30 f...",b'Chvez declares Angel Falls is no more: World...,b'Drug tests catch out 10 police (UK)',"b'Once the worlds fourth-largest lake, the Ara...",...,b'Pig farts spark Australia gas scare - I shit...,b'Bin Laden daughter flees to Saudi embassy in...,b'Traumatic Brain Injuries: Growing evidence ...,b'The Pakistan Supreme Court has ordered the P...,"b""Here's a video the Taliban released this mor...",b'Fireworks set off aboard airliner',"b'Five VA men may face terrorism charges, Paki...","b""Ayatollah Montazeri's Legacy: In death he m...",NaN,NaN
681,2011-04-21,1,"Director of ""Restrepo"" and Photographer Chris ...",Everyone within 20km of Fukushima will be forc...,Bahrain: Activist Zaynab Al-Khawaja's letter t...,"More than 80,000 people are claiming incapacit...",Libya: 'mission creep' claims as UK sends in m...,Japan considers banning access to evacuation zone,Chinese oil giant Sinopec has stopped exportin...,6.0-magnitude earthquake strikes off east coas...,...,Russian migration official fired in racism row...,Small amounts of radioactive iodine found in b...,"War photographer, Oscar-nominated film directo...",Effeminate boys are being sent to a special ca...,Japan Government Declares 12-Mile Area Around ...,TEPCO admitted Wednesday that nuclear fuel in ...,Fukushima evacuees face arrest if they return ...,Prince Charles wins some kind of a record,NaN,NaN


In [15]:
# Three datapoints have NAN values. Since these are news headlines, 
# replace with empty string '' to denote the absence of headlines for that day.

In [16]:
data.fillna(value='', inplace=True)

In [17]:
data.iloc[null_indices]

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
277,2009-09-15,1,b'The Church of Scientology won\'t be dissolve...,b'New virus from rats can kill 80 per cent of ...,b'The gruesome spectacle of dolphins being sla...,b'The End of Innocence in Afghanistan: \'The G...,b'France approves Internet piracy bill',b'The Rural Doctors Association says right now...,b'Al Jazeera English - Africa - Shabab to aven...,"b""How Sri Lanka governs through detentions - S...",...,b'In an equine echo of the controversy surroun...,b'UPDATE: 5-New York homes raided in terrorism...,b'Population Growth Impeding Progress on the M...,b'Global Population to Reach 7 Billion by 2011',b'Government Funded Feminist Porn ',b'Can someone enlighten me re:Holy Land disput...,b'Human Rights Watch official suspended for co...,,,
348,2009-12-24,1,b'Woman knocks down Pope Benedict at Christmas...,b'Ugandan President Museveni says he will bloc...,"b""Venezuela's Chavez threatens to kick out car...",b'Woman who knocked down pope had pasta in her...,"b'450 people from 21 countries, including 30 f...",b'Chvez declares Angel Falls is no more: World...,b'Drug tests catch out 10 police (UK)',"b'Once the worlds fourth-largest lake, the Ara...",...,b'Pig farts spark Australia gas scare - I shit...,b'Bin Laden daughter flees to Saudi embassy in...,b'Traumatic Brain Injuries: Growing evidence ...,b'The Pakistan Supreme Court has ordered the P...,"b""Here's a video the Taliban released this mor...",b'Fireworks set off aboard airliner',"b'Five VA men may face terrorism charges, Paki...","b""Ayatollah Montazeri's Legacy: In death he m...",,
681,2011-04-21,1,"Director of ""Restrepo"" and Photographer Chris ...",Everyone within 20km of Fukushima will be forc...,Bahrain: Activist Zaynab Al-Khawaja's letter t...,"More than 80,000 people are claiming incapacit...",Libya: 'mission creep' claims as UK sends in m...,Japan considers banning access to evacuation zone,Chinese oil giant Sinopec has stopped exportin...,6.0-magnitude earthquake strikes off east coas...,...,Russian migration official fired in racism row...,Small amounts of radioactive iodine found in b...,"War photographer, Oscar-nominated film directo...",Effeminate boys are being sent to a special ca...,Japan Government Declares 12-Mile Area Around ...,TEPCO admitted Wednesday that nuclear fuel in ...,Fukushima evacuees face arrest if they return ...,Prince Charles wins some kind of a record,,


In [18]:
# Check how many columns have an empty string in the entire dataframe.

## Preprocessing

### Structuring the data

All the 25 headlines for a day is combined into a single headline document.\
The date and label is also added into a new dataframe called as news.

In [20]:
documents = []
news = pd.DataFrame(data[data.columns[2:]].agg(' '.join, axis = 1))

In [21]:
news = pd.concat([data.Date, news, data.Label], axis = 1)
news.columns = ['Date', 'Text', 'Label']

In [157]:
# r"b\\\'|\\\'|b\"|\'nb|"

# When using or '|' in regex, start with the biggest pattern to match first followed by other smaller patters.

# For example, when r"b\'" and r"b" are patterns to be removed, the regex should be in the order r"b\'|b". 
# If not, the pattern r"b"  would be found first and the \' maybe ignored.

news.Text = news.Text.str.replace(pat=r"b\'|b\"|\'|\"|[.;:!\'?,\"()\[\]]", repl="", regex=True)
news.Text.iloc[0]

'Georgia downs two Russian warplanes as countries move to brink of war BREAKING Musharraf to be impeached Russia Today Columns of troops roll into South Ossetia footage from fighting YouTube Russian tanks are moving towards the capital of South Ossetia which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity UN official says - this is sick a three year old was raped and they do nothing 150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets Breaking Georgia invades South Ossetia Russia warned it would intervene on SOs side The enemy combatent trials are nothing but a sham Salim Haman has been sentenced to 5 1/2 years but will be kept longer anyway just because they feel like it Georgian troops retreat from S Osettain capital presumably leaving several hundred people killed VIDEO Did the US Prep Georgia for War with Russia Rice Gives Green Light for Israel to Attack Iran Says US has no veto over Israel

Shown above is the same text as shown before cleaned using regex.

### Text Preprocessing

Before preprocessing, the data is split into train and test. The vocabulary should be built only on the train data.

from 2008-08-08 to 2014-12-31 as Training Set, and Test Set is then the following two years data (from 2015-01-02 to 2016-07-01)

In [164]:
train = news[(news['Date'] >= pd.to_datetime('2008-08-08')) & (news['Date'] <= pd.to_datetime('2014-12-31'))]
test = news[(news['Date'] >= pd.to_datetime('2015-01-02')) & (news['Date'] <= pd.to_datetime('2016-07-01'))]

y_train = train.Label.to_list()
y_test = test.Label.to_list()

Three different preprocessing steps are defined. This is done to differentiate results in terms of preprocessing.

In [137]:
punctuations = string.punctuation.replace("", " ").split()
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

# Tokenization + Punctuation + Stop Words + Lemmatization
def preprocess_news_1(news):
    news = news.lower()
    tokens = nltk.word_tokenize(news)
    clean_words = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in punctuations and tok not in stopwords and len(tok) > 1]
    
    return ' '.join(clean_words)

# Tokenization + Punctuation + Lemmatization
def preprocess_news_2(news):
    news = news.lower()
    tokens = nltk.word_tokenize(news)
    clean_words = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in punctuations and len(tok) > 1]
    
    return ' '.join(clean_words)

# Tokenization + Punctuation + Stemming
def preprocess_news_3(news):
    news = news.lower()
    tokens = nltk.word_tokenize(news)
    clean_words = [ps.stem(tok) for tok in tokens if tok not in punctuations and len(tok) > 1]

    return ' '.join(clean_words)


In [153]:
preprocessing = [preprocess_news_1, preprocess_news_2, preprocess_news_3]

In [27]:
train_clean = train.Text.apply(preprocess_news).to_list()
X_train = [i.split() for i in train_clean]
y_train = train.Label.to_list()

test_clean = test.Text.apply(preprocess_news).to_list()
X_test = [i.split() for i in test_clean]
y_test = test.Label.to_list()

In [29]:
print(len(X_train), y_train.shape, len(X_test), y_test.shape)

1611 (1611,) 378 (378,)


## Word Embeddings

Every preprocessing method is used to train the following Word2Vec models:

    1. 64d Skipgram
    2. 64d CBOW
    3. 128d Skipgram
    4. 128d CBOW

In [163]:
def get_Word2Vec_models(X_train, preprocessing_index):
    
    model_64_sg = Word2Vec(sentences=X_train, size=64,
                           sg=1, window=10, iter=5,
                           min_count=2, workers=4)
    model_64_sg.wv.save_word2vec_format('word2vec_64_sg_' + str(preprocessing_index) +'.txt', binary=False)
    
    model_64_cbow = Word2Vec(sentences=X_train, size=64, 
                             sg=0, window=5, iter=5,
                             min_count=2, workers=4)
    model_64_cbow.wv.save_word2vec_format('word2vec_64_cbow_' + str(preprocessing_index) +'.txt', binary=False)
    
    model_128_sg = Word2Vec(sentences=X_train, size=128,
                            sg=1, window=10, iter=5,
                            min_count=2, workers=4)
    model_128_sg.wv.save_word2vec_format('word2vec_128_sg_' + str(preprocessing_index) +'.txt', binary=False)
    
    model_128_cbow = Word2Vec(sentences=X_train, size=128,
                              sg=0, window=5, iter=5,
                              min_count=2, workers=4)
    model_128_cbow.wv.save_word2vec_format('word2vec_128_cbow_' + str(preprocessing_index) +'.txt', binary=False)
    
    return [model_64_sg, model_64_cbow, model_128_sg, model_128_cbow]

In [36]:
print('Word in Vocabulary : ', len(model.wv.vocab))

Word in Vocabulary :  20826


In [37]:
model.wv.most_similar('obama')

[('barack', 0.9324471354484558),
 ('administration', 0.8562558889389038),
 ('chavez', 0.8426466584205627),
 ('obamas', 0.8256567120552063),
 ('bpresident', 0.8253248333930969),
 ('ahmadinejad', 0.8111671209335327),
 ('fm', 0.7826610803604126),
 ('admin', 0.7816630005836487),
 ('castro', 0.7809790968894958),
 ('chvez', 0.779906690120697)]

In [38]:
model.wv.doesnt_match("mother father sister brother dog".split())

D:\Anaconda\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'dog'

In [39]:
model.wv.doesnt_match("france england germany berlin".split())

'england'

In [40]:
model.wv.most_similar(positive=['husband', 'woman'], negative=['man'])

[('raped', 0.81412672996521),
 ('her', 0.8101285696029663),
 ('herself', 0.8048563003540039),
 ('rapist', 0.8044365644454956),
 ('honour', 0.8011881113052368),
 ('underage', 0.7964402437210083),
 ('pregnant', 0.7956597208976746),
 ('girl', 0.792364239692688),
 ('daughter', 0.7864694595336914),
 ('womans', 0.7848000526428223)]

## Feature Engineering for Traditional ML

For example, let us consider an input of 10 words and a 64d word2vec model.

A single input datapoint now consists of many words. For example, say 10 words. As a result of the Word2Vec model, every word is represented as a 64 dimension vector. Thus, we have ten 64 dimension vectors.

As an input to the ML algorithm, these vectors can be aggregated to a single form. Two ways of doing that are:

1. Mean of all the ten 64 dimension vectors to get a single 64 dimension vector.
2. Sum of all the ten 64 dimension vectors to get a single 64 dimension vector.

The function _get_WordEmbedding_features_ acheives the same

In [41]:
print('Total number of words in vocabulary along with shape : ', model.wv.vectors.shape)

Total number of words in vocabulary along with shape :  (20826, 64)


In [228]:
def get_WordEmbedding_features(list_of_reviews, model):
    vector_size = model.vector_size
    mean_paragraph_vector = []
    sum_paragraph_vector = []
    
    #[word for review in list_of_reviews for word in review if ]
    
    for i in range(len(list_of_reviews)):
        paragraph_vector = [model.wv[word] for word in list_of_reviews[i] if word in model.wv]
                
        mean_paragraph_vector.append(np.array(np.mean(np.array(paragraph_vector), axis = 0)))
        sum_paragraph_vector.append(np.array(np.sum(np.array(paragraph_vector), axis = 0)))
        
    return [np.array(mean_paragraph_vector), np.array(sum_paragraph_vector)]

In [ ]:
get_WordEmbedding_features(X_train, Word2Vec_models[0])

In [250]:
X_train_mean_paragraph_vector, X_train_sum_paragraph_vector = get_WordEmbedding_features(X_train, Word2Vec_models[0])
X_test_mean_paragraph_vector, X_test_sum_paragraph_vector = get_WordEmbedding_features(X_test, Word2Vec_models[0])

## Modelling

Algorithms from tradition ML models to DL models is applied in this section. The general structure of the flow is as follows:
    
For each preprocessing method:
>For each Word2Vec model:
>>For each learning algorithm:
>>>-Measure Performace 

In [222]:
def fit_logistic_regression(X_train, y_train):

    lr = LogisticRegression()
    lr_param_grid = {'solver' : ['lbfgs', 'sag', 'newton-cg', 'saga'],
                     'penalty' : ['l2'],
                     'C' : [0.05, 0.07, 0.09, 0.1, 0.5, 1],
                     'max_iter' : [500, 1000],
                    }

    lr_grid_search = GridSearchCV(estimator=lr, param_grid=lr_param_grid, n_jobs=-1)

    try:
        lr_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)
    
    print("\n Logistic Regression Over\n")
    return lr_grid_search

def fit_naive_bayes(X_train, y_train):
    
    NB = naive_bayes.GaussianNB()
    NB.fit(X_train, y_train)
    
    print("\n Naive Bayes Over\n")
    
    return NB

def fit_decision_tree(X_train, y_train):
    dt = tree.DecisionTreeClassifier()
    dt_param_grid = {'criterion' : ['gini', 'entropy'],
                     'max_depth' : [2, 4, 6, 8, 10, 12],
                     'min_samples_split' : [0.1, 0.2, 0.3, 0.4],
                     'min_samples_leaf' : [0.1, 0.2, 0.3, 0.4, 0.5],
                     'max_features' : ["sqrt", "auto", "log2"]
                    }

    dt_grid_search = GridSearchCV(dt, dt_param_grid, n_jobs = -1)

    try:
        dt_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)
    
    print("\n Decision Tree Over\n")
    
    return dt_grid_search
    
def fit_random_forest(X_train, y_train):
    rf = RandomForestClassifier()
    rf_param_grid = {'bootstrap': [True],
                     'criterion' : ["gini", "entropy"]
                  'max_depth': [None, 80, 90, 100, 110],
                  'max_features': [2, 3],
                  'min_samples_leaf': [1, 2, 3, 4, 5],
                  'min_samples_split': [2, 4, 6, 8, 10],
                  'n_estimators': [100, 200, 300, 1000]
                }
    , 
    rf_grid_search = GridSearchCV(rf, rf_param_grid, n_jobs = -1)

    try:
        rf_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)
        
    print("\n Random Forest Over\n")
    
    return rf_grid_search

def fit_SVM_nonlinear(X_train, y_train):
    
    SVM = SVC()
    svm_param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [10, 1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
    
    svm_grid_search = GridSearchCV(SVM, svm_param_grid, n_jobs = -1)

    try:
        svm_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)
        
    print("\n SVM Non-Linear Over\n")
    
    return svm_grid_search
  
def fit_svm_linear(X_train, y_train):
    
    # An SGDC with hinge loss is similar to a linear SVM.
    # These are regularized linear models and based on the loss function, the model is changed.
    SVM = SGDClassifier(loss='hinge', penalty='l2',
                        max_iter=5, tol=None)
    
    # A high value of alpha indicates a strong regularization.
    svm_param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
    
    svm_grid_search = GridSearchCV(SVM, svm_param_grid, n_jobs = -1)
    
    try:
        svm_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)
        
    print("\n Linear SVM Over\n")
    
    return svm_grid_search

In [243]:
# fit_naive_bayes has to be included.
learning_algorithms = [fit_logistic_regression, fit_decision_tree, fit_random_forest, fit_svm_linear, fit_SVM_nonlinear]

In [283]:
result = dict()
feat = ['mean', 'sum']

for index in range(len(preprocessing)):
    X_train = train.Text.apply(preprocessing[index]).to_list()
    X_test = test.Text.apply(preprocessing[index]).to_list()
    
    Word2Vec_models = get_Word2Vec_models(X_train, index+1)
    
    for W2V_model in Word2Vec_models: #have to check
        features = []
        features.append(get_WordEmbedding_features(X_train, W2V_model))
        features.append(get_WordEmbedding_features(X_test, W2V_model))
        
        for feature_index in range(len(features)):
            X_train_feature = features[0][feature_index]
            X_test_feature = features[1][feature_index]

            for algo in learning_algorithms:
                name = str(algo).split()[1][4:] + 'pre_' + str(index) + '_' + feat[feature_index]
                result[name] = {}
                fitted_model = algo(X_train_feature, y_train)
                try:
                    result[name].update({'best_params_' : fitted_model.best_params_})
                except:
                    print('Best Params Not for - ', name)
                result[name].update({'Train Score' : fitted_model.score(X_train_feature, y_train)})
                result[name].update({'Test Score' : fitted_model.score(X_test_feature, y_test)})
                result[name].update({'y_pred' : fitted_model.predict(X_test_feature)})
                try:
                    result[name].update({'y_predict_prob' : fitted_model.predict_proba(X_test_feature)[:,1]})
                except:
                    result[name].update({'y_predict_prob' : np.zeros(X_test_feature.shape)})
                result[name].update({'Accuracy' : accuracy_score(y_test, result[name]['y_pred'])})
                result[name].update({'Precision' : precision_score(y_test, result[name]['y_pred'])})
                result[name].update({'Recall' : recall_score(y_test, result[name]['y_pred'])})
                result[name].update({'roc_auc' : roc_auc_score(y_test, result[name]['y_pred'])})
                result[name].update({'Report' : classification_report(y_test, result[name]['y_pred'])})


 Logistic Regression Over



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Decision Tree Over



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Random Forest Over


 Linear SVM Over



...

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Random Forest Over


 Linear SVM Over



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 SVM Non-Linear Over



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [280]:
resultd_df = pd.DataFrame(result)
resultd_df.to_excel('ML Results.xlsx')

In [282]:
features

[[array([[ 0.0710889 ,  0.05101766, -0.11172073, ...,  0.08689123,
          -0.00728007,  0.06108397],
         [ 0.07129619,  0.05271976, -0.11299867, ...,  0.08634379,
          -0.00698603,  0.06252974],
         [ 0.0708759 ,  0.0475807 , -0.10929758, ...,  0.08642882,
          -0.00709747,  0.06152684],
         ...,
         [ 0.0722062 ,  0.04616151, -0.11043011, ...,  0.0870188 ,
          -0.00856543,  0.06124606],
         [ 0.07240389,  0.04501645, -0.10937918, ...,  0.0867654 ,
          -0.00895415,  0.06008755],
         [ 0.07020044,  0.04963039, -0.11039422, ...,  0.086511  ,
          -0.00861241,  0.06015614]], dtype=float32),
  array([[ 120.56677 ,   86.525955, -189.47836 , ...,  147.36752 ,
           -12.347002,  103.598404],
         [  83.202644,   61.523956, -131.86945 , ...,  100.7632  ,
            -8.152701,   72.972206],
         [ 106.38472 ,   71.41863 , -164.05566 , ...,  129.72966 ,
           -10.653307,   92.35178 ],
         ...,
         [ 123.4726

### Traditional ML Models

#### Logistic Regression

In [252]:
#'penalty' : ['l2', 'l1', 'elasticnet'],
#'liblinear'

def ffit_logistic_regression(X_train, y_train):

    lr = LogisticRegression()

    lr_param_grid = {'solver' : ['lbfgs', 'sag', 'newton-cg', 'saga'],
                     'penalty' : ['l2'],
                     'C' : [0.06, 0.07, 0.08, 0.09, 0.1, 1],
                     'max_iter' : [500, 1000],
                    }

    lr_grid_search = GridSearchCV(estimator=lr, param_grid=lr_param_grid)

    try:
        lr_grid_search.fit(X_train, y_train)
    except Exception as e:
        print(e)

    return lr_grid_search

In [253]:
lr_mean_embedding = ffit_logistic_regression(X_train_mean_paragraph_vector, y_train)

D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
D:\Anaconda\l

In [258]:
print(lr_mean_embedding.best_params_)


{'C': 0.06, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
0.5418994413407822


In [279]:
#print(lr_mean_embedding.best_estimator_.score(X_train_mean_paragraph_vector, y_train))
print(lr_mean_embedding.score(X_test_mean_paragraph_vector, y_test))

0.5079365079365079


In [260]:
lr_sum_embedding = fit_logistic_regression(X_train_sum_paragraph_vector, y_train)


 Logistic Regression Over



D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [261]:
lr_sum_embedding.best_params_

{'C': 0.05, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

In [263]:
lr_sum_embedding.score(X_test_sum_paragraph_vector, y_test)

0.5158730158730159

In [48]:
print(lr_mean_embedding.best_params_)
print(lr_sum_embedding.best_params_)

{'C': 0.06, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
{'C': 1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}


In [49]:
print('Logistic Regression Results \n')
print(lr_mean_embedding.best_estimator_.score(X_train_mean_paragraph_vector, y_train))
print(lr_mean_embedding.best_estimator_.score(X_test_mean_paragraph_vector, y_test))

print(lr_sum_embedding.best_estimator_.score(X_train_sum_paragraph_vector, y_train))
print(lr_sum_embedding.best_estimator_.score(X_test_sum_paragraph_vector, y_test))

Logistic Regression Results 

0.5418994413407822
0.5079365079365079
0.5822470515207946
0.5026455026455027


#### Naive Bayes

In [50]:
from sklearn import naive_bayes
NB = naive_bayes.GaussianNB()

In [51]:
NB_mean = NB.fit(X_train_mean_paragraph_vector, y_train)
NB_sum = NB.fit(X_train_sum_paragraph_vector, y_train)

In [52]:
print('Naive Bayes Results \n')
print(NB_mean.score(X_train_mean_paragraph_vector, y_train))
print(NB_mean.score(X_test_mean_paragraph_vector, y_test))

print(NB_mean.score(X_train_sum_paragraph_vector, y_train))
print(NB_mean.score(X_test_sum_paragraph_vector, y_test))

Naive Bayes Results 

0.4581005586592179
0.49206349206349204
0.5667287399130975
0.4947089947089947


#### Random Forest

In [53]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(n_estimators=100, criterion='entropy')

In [54]:
RF_mean = clf_RF.fit(X_train_mean_paragraph_vector, y_train)
RF_sum = clf_RF.fit(X_train_sum_paragraph_vector, y_train)
# Predicting the sentiment values for test data and saving the results in a csv file 
#result = forest.predict(X_test_mean_paragraph_vector)

In [55]:
print('Random Forest Mean Embedding Resutls\n')
print(RF_mean.score(X_train_mean_paragraph_vector, y_train))
print(RF_mean.score(X_test_mean_paragraph_vector, y_test))

print('\nRandom Forest Sum Embedding Resutls\n')
print(RF_sum.score(X_train_sum_paragraph_vector, y_train))
print(RF_sum.score(X_test_sum_paragraph_vector, y_test))

Random Forest Mean Embedding Resutls

0.5418994413407822
0.5079365079365079

Random Forest Sum Embedding Resutls

1.0
0.5


### Deep Learning Models

Making some checks before setting embedding layer parameters

#### Using Word2Vec Models created

In [116]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

max_length = max([len(headline) for headline in X_train])
vocab_size = len(tokenizer.word_index) + 1

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [117]:
def load_embedding_model(model_name):
    file = open(model_name,'r')
    lines = file.readlines()[1:] #Remove the first line as it contains vocabulary size and dimension
    file.close()
    
    embedding_mapper = dict() #To map words with their vectors
    for line in lines:
        word_vector = line.split()
        embedding_mapper[word_vector[0]] = np.array(word_vector[1:], dtype='float32')
    
    return embedding_mapper

def get_tokenizer_embedding_mapper(embedding, tokenizer_word_index):
    not_present = []
    vocab_size = len(tokenizer_word_index) + 1
    tokenizer_embedding_mapper = np.zeros((vocab_size, model.wv.vector_size))
    for word, index in tokenizer_word_index.items():
        try:
            tokenizer_embedding_mapper[index] = embedding[word]
        except:
            not_present.append(word)
            
    return tokenizer_embedding_mapper, not_present

In [118]:
learned_embedding = load_embedding_model('embedding64_word2vec.txt')

tokenizer_mapper, oov = get_tokenizer_embedding_mapper(learned_embedding, tokenizer.word_index)

In [120]:
# This shows that we have succesfully mapped all words learnt by our Word2Vec model.
len(oov) + len(learned_embedding) == len(tokenizer.word_index)

True

In [121]:
vocab_size

40453

In [133]:
embedding_layer = Embedding(vocab_size, model.wv.vector_size, weights=[tokenizer_mapper], input_length=max_length, trainable=False)

#### Dense Neural Network

In [134]:
MLP = Sequential()
MLP.add(embedding_layer)
MLP.add(Flatten())
MLP.add(Dense(64, activation='relu'))
MLP.add(Dropout(0.5))
# model.add(Dense(n_dense, activation='relu'))
# model.add(Dropout(dropout))
MLP.add(Dense(1, activation='sigmoid'))
MLP.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 671, 64)           2588992   
_________________________________________________________________
flatten_2 (Flatten)          (None, 42944)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2748480   
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 5,337,537
Trainable params: 2,748,545
Non-trainable params: 2,588,992
_________________________________________________________________


In [139]:
MLP1 = Sequential()
MLP1.add(embedding_layer)
MLP1.add(Flatten())
MLP1.add(Dense(128, activation='relu'))
MLP1.add(Dropout(0.2))
# model.add(Dense(n_dense, activation='relu'))
# model.add(Dropout(dropout))
MLP1.add(Dense(1, activation='sigmoid'))
MLP1.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 671, 64)           2588992   
_________________________________________________________________
flatten_3 (Flatten)          (None, 42944)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               5496960   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 8,086,081
Trainable params: 5,497,089
Non-trainable params: 2,588,992
_________________________________________________________________


In [140]:
MLP.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
MLP1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [141]:
MLP.fit(np.array(X_train_pad), np.array(y_train), 
          batch_size=64, epochs=5, verbose=1, 
          validation_data=(np.array(X_test_pad), np.array(y_test))
          )

Train on 1611 samples, validate on 378 samples
Epoch 1/5
1611/1611 [==============================] - 1s 653us/sample - loss: 0.2142 - accuracy: 0.8926 - val_loss: 0.9301 - val_accuracy: 0.5265
Epoch 2/5
1611/1611 [==============================] - 0s 208us/sample - loss: 0.1930 - accuracy: 0.9007 - val_loss: 0.9325 - val_accuracy: 0.5159
Epoch 3/5
1611/1611 [==============================] - 0s 209us/sample - loss: 0.1756 - accuracy: 0.9044 - val_loss: 0.9256 - val_accuracy: 0.4921
Epoch 4/5
1611/1611 [==============================] - 0s 202us/sample - loss: 0.1662 - accuracy: 0.9056 - val_loss: 1.0863 - val_accuracy: 0.4947
Epoch 5/5
1611/1611 [==============================] - 0s 204us/sample - loss: 0.1651 - accuracy: 0.8982 - val_loss: 1.0816 - val_accuracy: 0.4974


In [146]:
MLP1.fit(np.array(X_train_pad), np.array(y_train), 
          batch_size=32, epochs=4, verbose=1, 
          validation_data=(np.array(X_test_pad), np.array(y_test))
          )

Train on 1611 samples, validate on 378 samples
Epoch 1/4
1611/1611 [==============================] - 1s 772us/sample - loss: 0.0533 - accuracy: 0.9851 - val_loss: 1.1893 - val_accuracy: 0.5053
Epoch 2/4
1611/1611 [==============================] - 1s 611us/sample - loss: 0.0443 - accuracy: 0.9857 - val_loss: 1.0620 - val_accuracy: 0.4841
Epoch 3/4
1611/1611 [==============================] - 1s 597us/sample - loss: 0.0374 - accuracy: 0.9839 - val_loss: 1.3132 - val_accuracy: 0.5212
Epoch 4/4
1611/1611 [==============================] - 1s 593us/sample - loss: 0.0347 - accuracy: 0.9826 - val_loss: 1.0813 - val_accuracy: 0.5185


#### Convolutional Neural Network

In [193]:
CNN = Sequential()

CNN.add(embedding_layer)

CNN.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
CNN.add(GlobalMaxPooling1D())
CNN.add(Dropout(0.4))
CNN.add(Flatten())

CNN.add(Dense(32, activation='relu'))
CNN.add(Dropout(0.4))

CNN.add(Dense(1, activation='sigmoid'))

print(CNN.summary())

# compile network
CNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
CNN.fit(X_train_pad, y_train, epochs=15, verbose=2)
# evaluate
loss, acc = CNN.evaluate(X_test_pad, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 671, 64)           2588992   
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 667, 64)           20544     
_________________________________________________________________
global_max_pooling1d_33 (Glo (None, 64)                0         
_________________________________________________________________
dropout_70 (Dropout)         (None, 64)                0         
_________________________________________________________________
flatten_37 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_74 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_71 (Dropout)         (None, 32)              